In [12]:
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import IPython.display as ipd

# from keras.models import Sequential
# from keras.layers import Dense
from sklearn.preprocessing import MultiLabelBinarizer
# from sklearn.model_selection import RepeatedKFold
# from sklearn.metrics import accuracy_score

import soundfile as sf
from multiprocessing import Pool
# from tqdm import tqdm
import os


In [3]:
DEV_ROOT = './Data/FSD50K.dev_audio/'
TEST_ROOT = './Data/FSD50K.eval_audio/'

raw_dev_df = pd.read_csv('./Data/FSD50K.ground_truth/dev.csv', dtype={'fname': 'string'})

test_df = pd.read_csv('./Data/FSD50K.ground_truth/eval.csv', dtype={'fname': 'string'})
vocab_df = pd.read_csv('./Data/FSD50K.ground_truth/vocabulary.csv', index_col=0, header=None, names=['label','mids'])
vocab_dict = dict(zip(vocab_df.mids, vocab_df.label))

In [4]:
raw_dev_df.head()

,fname,labels,mids,split
0,64760,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf",train
1,16399,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf",train
2,16401,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf",train
3,16402,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf",train
4,16404,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf",train


### EDA

In [36]:
def audio_adder(filename):
    audio_file = DEV_ROOT + filename + '.wav'
    data, sample_rate = librosa.load(audio_file, sr = None)
    return(len(data)/sample_rate)

_Extracting file length takes a while because i/o_

In [37]:
# train_df['length'] = train_df['fname'].apply(lambda x: audio_adder(x))
# val_df['length'] = val_df['fname'].apply(lambda x: audio_adder(x))

In [38]:
train_df

,fname,labels,mids
0,64760,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf"
1,16399,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf"
2,16401,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf"
3,16402,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf"
4,16404,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf"
...,...,...,...
40961,102863,"Fowl,Livestock_and_farm_animals_and_working_an...","/m/025rv6n,/m/0ch8v,/m/0jbk"
40962,389607,"Fowl,Livestock_and_farm_animals_and_working_an...","/m/025rv6n,/m/0ch8v,/m/0jbk"
40963,90091,"Fowl,Livestock_and_farm_animals_and_working_an...","/m/025rv6n,/m/0ch8v,/m/0jbk"
40964,244718,"Fowl,Livestock_and_farm_animals_and_working_an...","/m/025rv6n,/m/0ch8v,/m/0jbk"


### Pre-processing for training

#### Input chunking for fixed length mfcc conversion

In [5]:
# temporary files list, change to use dataframe later
files_list = ['63','699']
lf = len(files_list)

In [6]:
def replicate(data, min_clip_len):
  if len(data) < min_clip_len:
    tile_size = (min_clip_len // data.shape[0]) + 1
    data = np.tile(data, tile_size)[:min_clip_len]
  return data

In [ ]:
# Saving so that we only need to do this once
PARTS_DIR = './Data/Chunks/'
def process_idx(idx):
  # print("in pool: "+str(idx))
  # return idx
  file_name = files_list[idx]
  file_path = DEV_ROOT + file_name + '.wav'
  data, sample_rate = librosa.load(file_path)
  min_clip_len = int(sample_rate * 1)
  parts = []
  if len(data) < min_clip_len:
    data = replicate(data, min_clip_len)
    parts.append(data)
  else:
    overlap = int(sample_rate * 0.5) # 50% overlap
    for ix in range(0, len(data), overlap):
      clip_ix = data[ix:ix+min_clip_len]
      clip_ix = replicate(clip_ix, min_clip_len)
      parts.append(clip_ix)
  ret = []
  for i in range(len(parts)):
    # path = os.path.join(PARTS_DIR, "{}_{:04d}.wav".format(file_name, i))
    # sf.write(path, parts[i], sample_rate, "PCM_16")
    y = parts[i]
    sr = sample_rate
    x = librosa.feature.mfcc(y=y,sr=sr).flatten()
    ret.append(np.array([file_name,x])) #need to mess around with this to get proper output
  return(ret)


In [ ]:
# Multithreaded processing to process multiple files at the same time
pool = Pool(2)
o = pool.map_async(process_idx, range(lf))
res = o.get()
pool.close()
pool.join()

-----

In [48]:
res

In [ ]:
def get_model(n_inputs, n_outputs):
	model = Sequential()
	model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(n_outputs, activation='sigmoid'))
	model.compile(loss='binary_crossentropy', optimizer='adam')
	return model

In [ ]:
def evaluate_model(X, y):
	results = list()
	n_inputs, n_outputs = X.shape[1], y.shape[1]
	cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
	for train_ix, test_ix in cv.split(X):
		X_train, X_test = X[train_ix], X[test_ix]
		y_train, y_test = y[train_ix], y[test_ix]
		model = get_model(n_inputs, n_outputs)
		model.fit(X_train, y_train, verbose=0, epochs=100)
		yhat = model.predict(X_test)
		yhat = yhat.round()
        
		acc = accuracy_score(y_test, yhat)
		print('>%.3f' % acc)
		results.append(acc)
	return results

In [29]:
test_df

,fname,labels,mids
0,37199,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf"
1,175151,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf"
2,253463,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf"
3,329838,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf"
4,1277,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf"
...,...,...,...
10226,266616,Livestock_and_farm_animals_and_working_animals...,"/m/0ch8v,/m/0jbk"
10227,137745,"Fowl,Livestock_and_farm_animals_and_working_an...","/m/025rv6n,/m/0ch8v,/m/0jbk"
10228,418509,"Fowl,Livestock_and_farm_animals_and_working_an...","/m/025rv6n,/m/0ch8v,/m/0jbk"
10229,76133,"Fowl,Livestock_and_farm_animals_and_working_an...","/m/025rv6n,/m/0ch8v,/m/0jbk"
